In [1]:
import sys
sys.path.insert(0, '..')

# Граф

In [24]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
                            event_name="event",
                            event_timestamp="timestamp",
                            user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

In [25]:
source_df = source.to_dataframe()
source_df['event_name'].value_counts()

catalog             14518
main                 5635
lost                 3098
cart                 2842
product2             2172
delivery_choice      1686
product1             1515
payment_choice       1107
delivery_courier      834
payment_done          706
payment_card          565
delivery_pickup       506
payment_cash          197
Name: event_name, dtype: int64

# Описание работы функции

Датапроцессор удаляет траектории пользователей, длина которого меньше заданных параметрами


Параметры:
```events_num``` - количество событий с которым сравнивается длина траектории каждого пользователя.
Длина траектории в этом случае - количество уникальных timestamp (так как мы считаем, что с одинаковым timestamp могут быть только синтетические события).
Удаляются траектории, длина которых строго меньше events_num
```cutoff``` - отсечка по времени с которой сравнивается длина траектории каждого пользователя.
Длина траектории в этом случае - разность между последним и первым событием для каждого пользователя, рассчитанная в заданных единицах времени.
Удаляются траектории, длина которых строго меньше cutoff


In [34]:
%%time
from src.data_processors_lib.rete import DeleteUsersByPathLength, DeleteUsersByPathLengthParams

params = {'cutoff': (10000, 'D')

}

graph = PGraph(source_stream=source)
delete_short_paths = EventsNode(
    DeleteUsersByPathLength(params=DeleteUsersByPathLengthParams(**params)))

graph.add_node(
    node=delete_short_paths,
    parents=[graph.root]
)

result = graph.combine(
    node=delete_short_paths
)
result.to_dataframe()

CPU times: user 300 ms, sys: 26.3 ms, total: 327 ms
Wall time: 359 ms


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,6eada169-3300-45ec-b278-4a2b9873f135,raw,0,catalog,2019-11-01 17:59:13.273932,219483890.0
1,db355dbf-fba6-46af-8263-8f97c291fc78,raw,1,product1,2019-11-01 17:59:28.459271,219483890.0
2,f0a46610-d4ce-4835-b8ce-d93b92571efb,raw,2,cart,2019-11-01 17:59:29.502214,219483890.0
3,a7eb71b2-41c9-4320-af0c-cba61bddfa8d,raw,3,catalog,2019-11-01 17:59:32.557029,219483890.0
4,ff877727-799d-43eb-aa2d-bef004ce526e,raw,4,catalog,2019-11-01 21:38:19.283663,964964743.0
...,...,...,...,...,...,...
54615,ccc9e3ea-284c-4f71-8301-46eb61e08de2,raw,54615,catalog,2020-04-29 12:47:40.975732,501098384.0
54616,ad291878-86ae-4a54-b928-ca13c9aa032b,raw,54616,catalog,2020-04-29 12:48:01.809577,501098384.0
54617,7fc78139-3914-4600-9106-88cc0a94dae8,raw,54617,main,2020-04-29 12:48:01.938488,501098384.0
54618,758bc0d3-1443-41d4-a5d9-59dd08f6c8d4,raw,54618,catalog,2020-04-29 12:48:06.595390,501098384.0
